In [ ]:
import numpy as np
import cv2 as cv
import os

En este script, utilizo las bibliotecas `numpy` y `cv2` para manejar operaciones de procesamiento de imágenes con OpenCV en Python. 

In [ ]:

rostro = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
cap = cv.VideoCapture(0)

output_folder = 'C:\\Users\\Janin\\Documents\\IA\\emociones\\felicidad2\\'
files = os.listdir(output_folder)
# Extrae los números de los nombres de archivos existentes y encuentra el máximo
if files:
    i = max([int(f.split('feliz')[1].split('.png')[0]) for f in files if 'feliz' in f]) + 1
else:
    i = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in rostros:
        frame2 = frame[y:y+h, x:x+w]
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_AREA)
        cv.imwrite(output_folder + 'feliz' + str(i) + '.png', frame2)
        i += 1
        
    cv.imshow('rostros', frame)
    k = cv.waitKey(1)
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()

### Script de detección de rostros con OpenCV

Desarrollé este script para capturar video en tiempo real usando OpenCV, aprovechando la cámara integrada. Implementé un clasificador Haar (haarcascade_frontalface_alt.xml) para identificar rostros en los fotogramas en tiempo real. Cada rostro detectado es recortado y redimensionado a 100x100 píxeles, y luego se guarda automáticamente en una carpeta específica con nombres secuenciales que comienzan con 'feliz'. El proceso se muestra en vivo y finaliza al presionar la tecla 'Esc'. Además, el script continua la numeración de archivos de forma secuencial basándose en archivos previamente guardados en la carpeta.


In [ ]:
import cv2
import os
import numpy as np
import time

def obtenerModelo(method, facesData, labels):
    if method == 'FisherFaces': 
        emotion_recognizer = cv2.face.FisherFaceRecognizer_create()

        # Entrenando el reconocedor de rostros
        print("Entrenando ( "+method+" )...")
        inicio = time.time()
        emotion_recognizer.train(facesData, np.array(labels))
        tiempoEntrenamiento = time.time()-inicio
        print("Tiempo de entrenamiento ( "+method+" ): ", tiempoEntrenamiento)

        # Almacenando el modelo obtenido
        emotion_recognizer.write("modelo"+method+".xml")

dataPath = 'C:\\Users\\Janin\\Documents\\IA\\emociones' # Cambia a la ruta donde hayas almacenado Data
emotionsList = os.listdir(dataPath)
print('Lista de emociones: ', emotionsList)

labels = []
facesData = []
label = 0

for nameDir in emotionsList:
    emotionsPath = os.path.join(dataPath, nameDir)

    for fileName in os.listdir(emotionsPath):
        labels.append(label)
        facesData.append(cv2.imread(os.path.join(emotionsPath, fileName), 0))

    label += 1

# Llamando a la función para entrenar y almacenar solo el modelo FisherFaces
obtenerModelo('FisherFaces', facesData, labels)


# Entrenamiento de reconocedor de emociones con FisherFaces

En este script, utilizo OpenCV para entrenar un reconocedor de emociones utilizando el método FisherFaces. Cargo todas las imágenes desde un directorio especificado que contiene imágenes etiquetadas por emociones. Después de leer las imágenes y asignar etiquetas, entreno el modelo utilizando los datos de las caras y las etiquetas correspondientes. El tiempo que toma entrenar el modelo se mide y se imprime. Finalmente, guardo el modelo entrenado en un archivo XML para su uso futuro. Este proceso ayuda a desarrollar un sistema capaz de reconocer emociones en imágenes faciales de manera eficiente.


In [ ]:
import cv2
import os
import numpy as np

# Método usado para el entrenamiento y lectura del modelo
method = 'FisherFaces'

# Crea el reconocedor de rostros según el método seleccionado
if method == 'FisherFaces':
    emotion_recognizer = cv2.face.FisherFaceRecognizer_create()

# Lee el modelo entrenado
emotion_recognizer.read('modelo' + method + '.xml')

dataPath = 'C:\\Users\\Janin\\Documents\\IA\\emociones'  # Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()

    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y + h, x:x + w]
        rostro = cv2.resize(rostro, (100, 100), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)

        cv2.putText(frame, '{}'.format(result), (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

        if method == 'FisherFaces':
            if result[1] < 70:
                cv2.putText(frame, '{}'.format(imagePaths[result[0]]), (x, y - 25), 2, 1.1, (0, 255, 0), 1,
                            cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:
                cv2.putText(frame, 'No identificado', (x, y - 20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imshow('Frame', frame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

# Detección en tiempo real 

En este script, implemento un sistema de reconocimiento facial en tiempo real utilizando la biblioteca OpenCV. Primero, cargo un modelo preentrenado de reconocimiento de emociones usando FisherFaces. Luego, inicio la captura de video en tiempo real usando la webcam. Para cada cuadro capturado, detecto rostros y aplico el modelo para predecir la emoción. Los resultados se muestran en la pantalla con un cuadro alrededor del rostro y una etiqueta que indica la emoción detectada o si el rostro no fue identificado claramente.


In [2]:
import cv2
import os
import numpy as np
import time

def obtenerModelo(method, facesData, labels, dataPath):
    if method == 'LBPH':
        emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()

        # Entrenando el reconocedor de rostros
        print("Entrenando ( "+method+" )...")
        inicio = time.time()
        emotion_recognizer.train(facesData, np.array(labels))
        tiempoEntrenamiento = time.time()-inicio
        print("Tiempo de entrenamiento ( "+method+" ): ", tiempoEntrenamiento)

        # Almacenando el modelo obtenido
        modelo_path = os.path.join(dataPath, "modelo"+method+".xml")
        emotion_recognizer.write(modelo_path)
        print("Modelo guardado en:", modelo_path)

dataPath = 'C:\\Users\\Janin\\Documents\\IA\\emociones'  # Cambia a la ruta donde hayas almacenado Data
emotionsList = os.listdir(dataPath)
print('Lista de emociones: ', emotionsList)

labels = []
facesData = []
label = 0

for nameDir in emotionsList:
    emotionsPath = os.path.join(dataPath, nameDir)

    for fileName in os.listdir(emotionsPath):
        labels.append(label)
        facesData.append(cv2.imread(os.path.join(emotionsPath, fileName), 0))
    label += 1

obtenerModelo('LBPH', facesData, labels, dataPath)


Lista de emociones:  ['asombro', 'felicidad', 'tristeza']
Entrenando ( LBPH )...
Tiempo de entrenamiento ( LBPH ):  16.691436767578125
Modelo guardado en: C:\Users\Janin\Documents\IA\emociones\modeloLBPH.xml


# Entrenamiento de reconocedor de emociones con LBPH

En este script, implementé un sistema para entrenar un reconocedor de emociones utilizando la técnica LBPH (Local Binary Patterns Histograms) de OpenCV. Después de configurar el ambiente y cargar las imágenes de un directorio especificado, procedí a entrenar el modelo con estos datos. El modelo entrenado se guarda en un archivo XML dentro del mismo directorio de las imágenes. Este proceso incluye la captura de tiempos para evaluar la eficiencia del entrenamiento.


In [3]:
import cv2
import os
import numpy as np

method = 'LBPH'

if method == 'LBPH': 
    emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()
    emotion_recognizer.read('modelo'+method+'.xml')

dataPath = 'C:\\Users\\Janin\\Documents\\IA\\emociones' # Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)

cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if ret == False: 
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()

    faces = faceClassif.detectMultiScale(gray,1.3,5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y+h,x:x+w]
        rostro = cv2.resize(rostro, (100, 100), interpolation=cv2.INTER_CUBIC)
        result = emotion_recognizer.predict(rostro)

        cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)

        # LBPH
        if method == 'LBPH':
            if result[1] < 60:
                cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
            else:
                cv2.putText(frame,'No identificado',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)

    cv2.imshow('Frame',frame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


imagePaths= ['asombro', 'felicidad', 'modeloLBPH.xml', 'tristeza']


# Detección en tiempo real 

En este script, implemento un sistema de reconocimiento facial en tiempo real utilizando la biblioteca OpenCV. Primero, cargo un modelo preentrenado de reconocimiento de emociones usando LBPH. Luego, inicio la captura de video en tiempo real usando la webcam. Para cada cuadro capturado, detecto rostros y aplico el modelo para predecir la emoción. Los resultados se muestran en la pantalla con un cuadro alrededor del rostro y una etiqueta que indica la emoción detectada o si el rostro no fue identificado claramente.